# Домашнее задание к лекции "Функции и работа с данными"

## Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по правилам:

*   оценка 2 и ниже — низкий рейтинг;
*   оценка 4 и ниже — средний рейтинг;
*   оценка 4.5 и 5 — высокий рейтинг.

Результат классификации запишите в столбец class.

In [3]:
import pandas as pd

data_1 = pd.read_csv('ratings.csv')
data_1.head()

def classify_movie(rating):
    if rating <= 2.0:
        return 'Low'
    elif rating <= 4.0:
        return 'Middle'
    elif 4.5 <= rating <= 5:
        return 'High'

data_1['rating_level'] = data_1['rating'].apply(classify_movie)
data_1.head()

,userId,movieId,rating,timestamp,rating_level
0,1,31,2.5,1260759144,Middle
1,1,1029,3.0,1260759179,Middle
2,1,1061,3.0,1260759182,Middle
3,1,1129,2.0,1260759185,Low
4,1,1172,4.0,1260759205,Middle


## Задание 2

Используйте файл keywords.csv.

Нужно написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определённому региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [4]:
geo_data = { 'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск'] }

data_2 = pd.read_csv('keywords.csv')
data_2.head()

def classify_geo(keyword, geo_data):
    for geo, info in geo_data.items():
        if set(keyword.lower().split(' ')) & set(info):
            return geo
    return 'undefined'

data_2['region'] = data_2['keyword'].apply(classify_geo, geo_data=geo_data)
data_2[data_2['region'] != 'undefined']

,keyword,shows,region
127,авито москва,979292,Центр
370,авито ру санкт петербург,425134,Северо-Запад
849,авито ярославль,209581,Центр
1063,фарпост владивосток,176951,Дальний Восток
1236,банк санкт петербург,174375,Северо-Запад
...,...,...,...
99292,афиша кино москва на сегодня,3704,Центр
99590,авито ярославль автомобили с пробегом,3673,Центр
99808,полармед мурманск запись на прием,3719,Северо-Запад
99890,яндекс метро москва,4548,Центр


## Задание 3

Есть мнение, что раньше снимали настоящее кино, не то что сейчас. Ваша задача — проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия: файл movies.csv и ratings.csv из базы. Нужно проверить, верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

Вы не будете затрагивать субьективные факторы выставления этих рейтингов, а пройдётесь по алгоритму:

1.   В переменную years запишите список из всех годов с 1950 по 2010 года.
2.   Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте алгоритм:

 1.   для каждой строки пройдите по всем годам списка years;
 2.   если номер года присутствует в названии фильма, то функция возвращает этот год, как год выпуска;
 3.   если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год.

3.   Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’.
4.   Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга.

In [6]:
data_mov = pd.read_csv('movies.csv')
data_mov.head()

data_rate = pd.read_csv('ratings.csv')
data_rate.head()

years = [i for i in range(1950, 2011)] # Создаем список years

def production_year(title, years): # Пишем функцию присваивания года выпуска фильма
    for year in years:
        if str(year) in title:
            return year
    return 1900

data_mov['year'] = data_mov['title'].apply(production_year, years=years) # Записываем год выпуска в столбец year

avg_rating = data_rate.groupby('movieId')['rating'].mean().reset_index() # Считаем средний рейтинг каждого фильма
merged = data_mov.merge(avg_rating, on='movieId') # Объединяем таблицы с фильмами и средними оценками
yearly_avg = merged.groupby('year')['rating'].mean().reset_index() # Считаем средний рейтинг фильмов по годам
yearly_avg.sort_values(by='rating', ascending=False) # Сортируем результат по убыванию

,year,rating
2,1951,3.906653
3,1952,3.802318
13,1962,3.772962
4,1953,3.738878
8,1957,3.693322
...,...,...
53,2002,3.191918
48,1997,3.183169
41,1990,3.166759
54,2003,3.154081


In [ ]:
# Написала альтернативную функцию присваивания года выпуска фильма через регулярку, чтобы не перебирать список

import re

def production_year_2(title):
    match = re.search(r'\((\d{4})\)', title)
    if match:
        year = int(match.group(1))
        if 1950 <= year <= 2010:
            return year
    return 1900

production_year_2('Toy Story (1945)')

1900